In [59]:
import cv2, os, sys
import pandas as pd
import json
import numpy as np
import random
from icecream import ic
from tqdm.auto import tqdm
import shutil as sh
from PIL import Image  
from PIL import Image  

from bbox_folder import bbox

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

## Concatenate all label csv files (if you have all label, ignore this step)

In [22]:
# root_directory = sys.path[0]
# file_path1 = os.path.join(root_directory, 'label_data', 'save_label_G.csv')
# file_path2 = os.path.join(root_directory, 'label_data', 'save_label_Z.csv')

# a = pd.read_csv(file_path1)
# b = pd.read_csv(file_path2)
# data_label = pd.concat([a,b])


,annotation_index,image_id,human_label,label_name
0,0,7057517,1,Dent
1,1,7073954,2,Other
2,2,7073954,2,Other
3,3,7074070,4,Scratch
4,4,7062766,4,Scratch
...,...,...,...,...
172,892,7044988,2,Other
173,893,7037292,3,Rim
174,894,7038343,4,Scratch
175,895,7038343,3,Rim


### Get id and file name

In [45]:
# root_directory = sys.path[0]  # current directory / data folder that includes Data: 'Images'&'Annotations'
# annotation_path = os.path.join(root_directory, 'Data', 'Annotations', 'annotated_functional_test3_fixed.json')
# data_image = bbox(annotation_path)
# data_image = pd.concat([data_image.iloc[:180,:], data_image.iloc[720:,:]])
# data_image

,file_name,image_id,id,x_min,y_min,x_max,y_max,width,height
0,8f27c5f1-3ea3-4392-9bc8-585c76057cd1/858a0246-...,7057517,504548,1050,649,1115,701,65,52
1,f58a99ab-350a-41df-8dbf-889326378c4a/858a0246-...,7073954,504081,1589,534,1615,571,26,37
2,f58a99ab-350a-41df-8dbf-889326378c4a/858a0246-...,7073954,504082,1354,453,1390,480,36,27
3,f58a99ab-350a-41df-8dbf-889326378c4a/e84bd41d-...,7074070,504086,902,1084,978,1141,76,57
4,b44a2093-2861-402b-96bd-23cac6359b94/e84bd41d-...,7062766,504658,1324,781,1421,884,97,103
...,...,...,...,...,...,...,...,...,...
892,5acc593f-c057-46db-81e5-3d657ddeae6b/3ab29178-...,7044988,504436,5,683,143,796,138,113
893,39795355-5bde-47a1-8e5e-ec67b86801af/be80b6a0-...,7037292,504309,0,429,300,1069,300,640
894,3e194ab1-9afa-4b28-aab4-48216990b6d6/fe18c3ff-...,7038343,504352,835,641,1296,1021,461,380
895,3e194ab1-9afa-4b28-aab4-48216990b6d6/fe18c3ff-...,7038343,504353,152,608,705,1180,553,572


In [81]:
data = pd.concat([data_image[['file_name', 'id']].reset_index(), data_label[['human_label','label_name']].reset_index()], axis=1)
data = data.drop(columns='index')
data

,file_name,id,human_label,label_name
0,8f27c5f1-3ea3-4392-9bc8-585c76057cd1/858a0246-...,504548,1,Dent
1,f58a99ab-350a-41df-8dbf-889326378c4a/858a0246-...,504081,2,Other
2,f58a99ab-350a-41df-8dbf-889326378c4a/858a0246-...,504082,2,Other
3,f58a99ab-350a-41df-8dbf-889326378c4a/e84bd41d-...,504086,4,Scratch
4,b44a2093-2861-402b-96bd-23cac6359b94/e84bd41d-...,504658,4,Scratch
...,...,...,...,...
352,5acc593f-c057-46db-81e5-3d657ddeae6b/3ab29178-...,504436,2,Other
353,39795355-5bde-47a1-8e5e-ec67b86801af/be80b6a0-...,504309,3,Rim
354,3e194ab1-9afa-4b28-aab4-48216990b6d6/fe18c3ff-...,504352,4,Scratch
355,3e194ab1-9afa-4b28-aab4-48216990b6d6/fe18c3ff-...,504353,3,Rim


## has already get all labels in one csv file

In [128]:
# change your label path
root_directory = sys.path[0]
file_path = os.path.join(root_directory, 'label_data', 'save_label_H.csv')
data_label = pd.read_csv(file_path)
data_label

,annotation_index,image_id,human_label,label_name
0,0,7057517,2,Other
1,1,7073954,2,Other
2,2,7073954,1,Dent
3,3,7074070,4,Scratch
4,4,7062766,1,Dent
...,...,...,...,...
892,892,7044988,4,Scratch
893,893,7037292,3,Rim
894,894,7038343,4,Scratch
895,895,7038343,3,Rim


In [129]:
root_directory = sys.path[0]  # current directory / data folder that includes Data: 'Images'&'Annotations'
annotation_path = os.path.join(root_directory, 'Data', 'Annotations', 'annotated_functional_test3_fixed.json')
data_image = bbox(annotation_path)
data_image

,file_name,image_id,id,x_min,y_min,x_max,y_max,width,height
0,8f27c5f1-3ea3-4392-9bc8-585c76057cd1/858a0246-...,7057517,504548,1050,649,1115,701,65,52
1,f58a99ab-350a-41df-8dbf-889326378c4a/858a0246-...,7073954,504081,1589,534,1615,571,26,37
2,f58a99ab-350a-41df-8dbf-889326378c4a/858a0246-...,7073954,504082,1354,453,1390,480,36,27
3,f58a99ab-350a-41df-8dbf-889326378c4a/e84bd41d-...,7074070,504086,902,1084,978,1141,76,57
4,b44a2093-2861-402b-96bd-23cac6359b94/e84bd41d-...,7062766,504658,1324,781,1421,884,97,103
...,...,...,...,...,...,...,...,...,...
892,5acc593f-c057-46db-81e5-3d657ddeae6b/3ab29178-...,7044988,504436,5,683,143,796,138,113
893,39795355-5bde-47a1-8e5e-ec67b86801af/be80b6a0-...,7037292,504309,0,429,300,1069,300,640
894,3e194ab1-9afa-4b28-aab4-48216990b6d6/fe18c3ff-...,7038343,504352,835,641,1296,1021,461,380
895,3e194ab1-9afa-4b28-aab4-48216990b6d6/fe18c3ff-...,7038343,504353,152,608,705,1180,553,572


In [130]:
data = pd.concat([data_image[['file_name', 'id']].reset_index(), data_label[['human_label','label_name']].reset_index()], axis=1)
data = data.drop(columns='index')
data

,file_name,id,human_label,label_name
0,8f27c5f1-3ea3-4392-9bc8-585c76057cd1/858a0246-...,504548,2,Other
1,f58a99ab-350a-41df-8dbf-889326378c4a/858a0246-...,504081,2,Other
2,f58a99ab-350a-41df-8dbf-889326378c4a/858a0246-...,504082,1,Dent
3,f58a99ab-350a-41df-8dbf-889326378c4a/e84bd41d-...,504086,4,Scratch
4,b44a2093-2861-402b-96bd-23cac6359b94/e84bd41d-...,504658,1,Dent
...,...,...,...,...
892,5acc593f-c057-46db-81e5-3d657ddeae6b/3ab29178-...,504436,4,Scratch
893,39795355-5bde-47a1-8e5e-ec67b86801af/be80b6a0-...,504309,3,Rim
894,3e194ab1-9afa-4b28-aab4-48216990b6d6/fe18c3ff-...,504352,4,Scratch
895,3e194ab1-9afa-4b28-aab4-48216990b6d6/fe18c3ff-...,504353,3,Rim


## Process images (gray, resize), 0-dim as list, preparing for the train process

In [131]:
resize_width = 255
resize_height = 236

image_array = []
image_label = []
for i,vlaue in data.iterrows():

    file_name, id, human_label, label_name = vlaue
    _, ext = os.path.splitext(file_name) 
    name = str(id)+ext
    name = os.path.join(root_directory, 'bbox_image', name)

    img = cv2.imread(name, cv2.IMREAD_GRAYSCALE)  # height, width
    img = cv2.resize(img, (resize_width, resize_height))
    img_array = img.flatten()

    image_array.append(img_array)
    image_label.append(human_label)

## Train 

In [142]:
x_train, x_test, y_train, y_test = train_test_split(image_array, image_label, random_state=42, test_size=0.3)
transfer = StandardScaler()
x_train = transfer.fit_transform(x_train)
x_test = transfer.transform(x_test)

estimator = KNeighborsClassifier(n_neighbors=4)
estimator.fit(x_train, y_train)

y_predict = estimator.predict(x_test)
print('y_predict:\n', y_predict)
# print('compare ground truth with predict value:\n', y_test == y_predict)

score = estimator.score(x_test, y_test)
print('accuracy:\n', score)


y_predict:
 [1 1 1 1 1 1 3 3 1 1 4 1 4 1 4 1 1 2 4 1 3 1 4 1 1 1 1 4 1 4 2 4 1 1 4 3 1
 4 4 4 1 4 4 1 1 3 1 1 4 1 2 1 1 4 1 1 4 4 4 4 2 4 1 2 4 1 1 4 1 1 4 4 4 4
 1 1 4 4 4 4 4 4 1 4 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 4 4 3 1 1 1 1 1 1
 4 4 1 1 4 1 4 4 1 1 1 1 1 1 4 4 4 1 4 1 1 1 1 4 4 1 4 4 4 4 4 4 1 4 1 4 2
 1 1 1 1 4 4 1 1 4 1 1 1 4 1 2 1 4 1 1 4 1 1 4 4 1 4 1 4 1 4 1 1 4 1 1 4 1
 2 1 1 2 1 1 4 4 1 1 4 1 1 3 4 4 1 1 4 4 4 1 1 1 4 1 1 1 1 1 3 1 4 1 4 4 4
 1 4 1 4 1 4 1 4 4 1 1 1 4 1 1 1 1 1 2 4 4 1 1 1 1 4 1 4 4 1 1 1 1 1 4 4 1
 2 1 1 4 3 1 3 1 4 4 4]
accuracy:
 0.4444444444444444
